# Data wrangling for *Mtb*-Tn-Mat project

* reads in SI tables / files and compiles them into Pandas dataframe 
* version 2.0: goal is to have 3 matrices:
    1. log2FC matrix
    2. q-value matrix

## Imports modules:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from statsmodels.stats.multitest import multipletests
from Tn_data_wrangling import *

%load_ext autoreload
%autoreload 2

pd.options.mode.chained_assignment = None  # default='warn

#### Read in full set of Rv_IDs and gene_names from tuberculist annotation: 

In [2]:
root_dir = '../../'
data_path = os.path.join(root_dir, 'data')

In [3]:
file_WG = 'annotations/H37Rv_tuberculist_functional_categories.xlsx'
df_WG = pd.read_excel(os.path.join(data_path, file_WG))
df_WG = df_WG[['Rv_ID', 'gene_name']]
df_WG.head(1)

,Rv_ID,gene_name
0,Rv0001,dnaA


## The first set of cases:

These are the SI files and tables (19 columns) that are in the following format:

they list the set of genes that are called (conditionally) essential in the Tn screen

In [4]:
# read in path/file_name to column name mapping for 'simple' cases:
col_name_file = os.path.join( data_path, 'SI_datasets/files_and_columns.csv') 
df_col_info_set1 = pd.read_csv(col_name_file)
df_col_info_set1

,file,col_name,q_val_col_name,ratio_col_name,is_ratio_log2FC
0,2003_Sassetti_Boyd_Rubin/table_1.xls,2003A_Sassetti,p_val,ratio,FALSE
1,2003_Sassetti_Rubin/table_3.xls,2003B_Sassetti,Week_8_P_value,Week_8_Ratio,FALSE
2,2005_Rengarajan_Rubin/table_2.xls,2005_Rengarajan,Nan,Nan,Nan
3,2006_Joshi_Sassetti/table_1.xlsx,2006_Joshi_GI_1,Nan,Nan,Nan
4,2006_Joshi_Sassetti/table_2.xlsx,2006_Joshi_GI_2,Nan,Nan,Nan
5,2011_Griffin_Sassetti/table_4.xlsx,2011_Griffin_2,p value,ratio,FALSE
6,2013_Zhang_Rubin/table_2.xlsx,2013_Zhang_1A,FDR (q-val) Day 10,in vitro:in vivo ratio Day 10,FALSE
7,2013_Zhang_Rubin/table_2.xlsx,2013_Zhang_1B,FDR (q-val) Day 45,in vitro:in vivo ratio Day 45,FALSE
8,2013_Zhang_Rubin/table_3.xlsx,2013_Zhang_2,MWU P-value,wt:MHC ratio,FALSE
9,2013_Zhang_Rubin/table_4A.xlsx,2013_Zhang_3A,MWU P-value,Ty6.5:Ty4.5 ratio,FALSE


#### Building q-value and log2FC matrix for first set of files: 

In [5]:
tn_data_path = os.path.join(root_dir, 'data/SI_datasets')
tn_data_path

'../../data/SI_datasets'

In [6]:
counter = 0

for index, row in df_col_info_set1.iterrows():
    
    file_in = row['file']
    col_name = row['col_name']
    qval_col = row['q_val_col_name']
    log2FC_col = row['ratio_col_name']
    is_ratio_log2FC = row['is_ratio_log2FC']
    target_qval_col = col_name+'_q_val'
    target_log2FC_col = col_name+'_log2FC'
    
    print(file_in, col_name)
    
    # get column names (both from SI file and target column names in Tn-Matrix )
#     qval_col, log2FC_col, is_ratio_log2FC, target_qval_col, target_log2FC_col = get_col_names(file_in, df_col_info_set1, col_name)
    
    # get data from SI file and populate dataframe. 
    df_tn_qval_log2FC = build_q_val_ratio_df( os.path.join(tn_data_path, file_in), col_name, qval_col, log2FC_col, is_ratio_log2FC )
    
    # merge with whole genome (Rv-IDs and gene-names for all of H37Rv)
    df_tn_qval_log2FC_WG = merge_with_whole_genome_qval_log2FC( df_WG, target_qval_col, target_log2FC_col, df_tn_qval_log2FC )

    # merge with rest of data-files / columns. 
    if counter == 0:
        df_tn_qval_log2FC_ALL = df_tn_qval_log2FC_WG.copy()
    else:
        df_tn_qval_log2FC_ALL = df_tn_qval_log2FC_ALL.merge(df_tn_qval_log2FC_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
    
    counter +=1

2003_Sassetti_Boyd_Rubin/table_1.xls 2003A_Sassetti
2003_Sassetti_Rubin/table_3.xls 2003B_Sassetti
2005_Rengarajan_Rubin/table_2.xls 2005_Rengarajan
2006_Joshi_Sassetti/table_1.xlsx 2006_Joshi_GI_1
2006_Joshi_Sassetti/table_2.xlsx 2006_Joshi_GI_2
2011_Griffin_Sassetti/table_4.xlsx 2011_Griffin_2
2013_Zhang_Rubin/table_2.xlsx 2013_Zhang_1A
2013_Zhang_Rubin/table_2.xlsx 2013_Zhang_1B
2013_Zhang_Rubin/table_3.xlsx 2013_Zhang_2
2013_Zhang_Rubin/table_4A.xlsx 2013_Zhang_3A
2013_Zhang_Rubin/table_4B.xlsx 2013_Zhang_3B
2013_Zhang_Rubin/table_4C.xlsx 2013_Zhang_3C
2013_Zhang_Rubin/table_4D.xlsx 2013_Zhang_3D
2016_Korte_Kalscheuer/table_1.xlsx 2016_Korte
2018_Rittershaus_Sassetti/table_2A.xlsx 2018_Rittershaus_1A
2018_Rittershaus_Sassetti/table_2B.xlsx 2018_Rittershaus_1B


/Users/ajinich/Documents/git_repos/mtb_tn_db/code/data_wrangling/Tn_data_wrangling.py:43: RuntimeWarning: divide by zero encountered in log2
  df_tn[target_log2FC_col] = np.log2(df_tn[log2FC_col])


### Zhang log2FC values are wrong. They report in vitro / in vivo

In [7]:
cols_to_invert = ['2013_Zhang_1A_log2FC', '2013_Zhang_1B_log2FC', '2013_Zhang_2_log2FC' ]

In [8]:
for col in cols_to_invert:
    df_tn_qval_log2FC_ALL[col] = -df_tn_qval_log2FC_ALL[col]

In [9]:
df_tn_qval_log2FC_ALL[['Rv_ID', '2013_Zhang_1B_q_val', '2013_Zhang_1B_log2FC']].head(10)

,Rv_ID,2013_Zhang_1B_q_val,2013_Zhang_1B_log2FC
0,Rv0001,NaN,NaN
1,Rv0002,NaN,NaN
2,Rv0003,0.000087,-3.223423
3,Rv0004,NaN,NaN
4,Rv0005,NaN,NaN
5,Rv0006,NaN,NaN
6,Rv0007,0.000007,-8.927778
7,Rv0008c,NaN,NaN
8,Rv0009,0.000725,-4.461704
9,Rv0010c,NaN,NaN


## Second set of cases: 

These are SI datafiles and tables that have, for all genes in H37Rv, a column for q-vals and ratios.

Load spreasheet with file name-to-column mappings

In [10]:
col_name_file = os.path.join( data_path, 'SI_datasets/files_and_columns_set2.csv') 
df_col_info_set2 = pd.read_csv(col_name_file)
df_col_info_set2

,file,col_name,q_val_col_name,ratio_col_name,is_ratio_log2FC
0,2012_Zhang_Rubin/table_4_sheet_600bp.xlsx,2012_Zhang,p-val avg,NONE,NONE
1,2015_Kieser_Rubin/table_1A_edited.xlsx,2015_Kieser_GI_1,P-value,Fold Change,FALSE
2,2015_Kieser_Rubin/table_1B_edited.xlsx,2015_Kieser_GI_2,P-value,Fold Change,FALSE
3,2015_Kieser_Rubin/table_1C_edited.xlsx,2015_Kieser_GI_3,P-value,Fold Change,FALSE
4,2015_Mendum_Stewart/table_1A.xlsx,2015_Mendum,p_val_7days,NONE,NONE
5,2016_Nambi_Sassetti/table_1.xlsx,2016_Nambi,Qval,Log2 ctpC/WT,TRUE
6,2017_Mishra_Sassetti/table_1.xlsx,2017_Mishra_1,Qval (C3HeBvsBL6),log2(C3HeB/BL6),TRUE
7,2017_Mishra_Sassetti/table_1.xlsx,2017_Mishra_2,Qval (NOS2vsBL6),log2(NOS2/BL6),TRUE
8,2017_Xu_Ehrt/table_3A.xlsx,2017_Xu_1A,qval,FC [log2],TRUE
9,2017_Xu_Ehrt/table_3B.xlsx,2017_Xu_1B,qval,FC [log2],TRUE


## Don't do the last two; they are Michael's and in a different format: 

In [51]:
df_col_info = df_col_info_set2[:-2].copy()

In [52]:
df_tn_qval_log2FC_ALL = get_qval_log2FC_func(df_col_info, df_WG, df_tn_qval_log2FC_ALL)

2012_Zhang_Rubin/table_4_sheet_600bp.xlsx 	 2012_Zhang
2015_Kieser_Rubin/table_1A_edited.xlsx 	 2015_Kieser_GI_1


/home/adrian/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


2015_Kieser_Rubin/table_1B_edited.xlsx 	 2015_Kieser_GI_2
2015_Kieser_Rubin/table_1C_edited.xlsx 	 2015_Kieser_GI_3
2015_Mendum_Stewart/table_1A.xlsx 	 2015_Mendum
2016_Nambi_Sassetti/table_1.xlsx 	 2016_Nambi
2017_Mishra_Sassetti/table_1.xlsx 	 2017_Mishra_1
2017_Mishra_Sassetti/table_1.xlsx 	 2017_Mishra_2
2017_Xu_Ehrt/table_3A.xlsx 	 2017_Xu_1A
2017_Xu_Ehrt/table_3B.xlsx 	 2017_Xu_1B
2017_Xu_Ehrt/table_3C.xlsx 	 2017_Xu_1C
2017_Xu_Ehrt/table_3D.xlsx 	 2017_Xu_1D
2017_Xu_Ehrt/table_3E.xlsx 	 2017_Xu_1E
2018_Carey_Fortune/table_1A.xlsx 	 2018_Carey_1A
2018_Carey_Fortune/table_1B.xlsx 	 2018_Carey_1B
2018_Carey_Fortune/table_1C.xlsx 	 2018_Carey_1C
2018_Carey_Fortune/table_1D.xlsx 	 2018_Carey_1D
2018_Carey_Fortune/table_1E.xlsx 	 2018_Carey_1E
2018_Carey_Fortune/table_1F.xlsx 	 2018_Carey_1F
2018_Carey_Fortune/table_1G.xlsx 	 2018_Carey_1G
2018_Carey_Fortune/table_1H.xlsx 	 2018_Carey_1H
2017B_DeJesus_Iorger/table_1A.xlsx 	 2017B_DeJesus_1A
2017B_DeJesus_Iorger/table_1B.xlsx 	 2017B_D

In [53]:
df_tn_qval_log2FC_ALL.shape

(3990, 112)

In [54]:
data_path

'../../data'

In [55]:
path_out = '../../data/SI_datasets/'
out_file = os.path.join(path_out, 'SI_qval_log2FC.xlsx')
df_tn_qval_log2FC_ALL.to_excel(out_file, index=False)

## checking a couple of gene profiles: 

## grabbing only 'full' data columns for Anisha's dash site: 

In [30]:
cols_set_2 = df_col_info_set2.col_name.values

In [31]:
cols_set_2_qvals_temp = [col+'_q_val' for col in cols_set_2]
cols_set_2_qvals = [col for col in cols_set_2_qvals_temp if col in df_tn_qval_log2FC_ALL.columns]


cols_set_2_log2FC_temp = [col+'_log2FC' for col in cols_set_2]
cols_set_2_log2FC = [col for col in cols_set_2_log2FC_temp if col in df_tn_qval_log2FC_ALL.columns]


In [32]:
df_tn_qval_set2 = df_tn_qval_log2FC_ALL[ ['Rv_ID', 'gene_name'] + cols_set_2_qvals]
df_tn_log2FC_set2 = df_tn_qval_log2FC_ALL[ ['Rv_ID', 'gene_name'] + cols_set_2_log2FC]


In [33]:
file_qval = os.path.join(data_path, 'Tn_library_qvals_DASH.xlsx')
df_tn_qval_set2.to_excel(file_qval, index=False)

file_log2FC = os.path.join(data_path, 'Tn_library_log2FC_DASH.xlsx')
df_tn_log2FC_set2.to_excel(file_log2FC, index=False)

### [PENDING] 

### DeJesus and Iorger (2013)

In [38]:
file = '2013_DeJesus_Iorger/table_1.xls'
col_name = '2013_DeJesus'
df_tn = pd.read_excel(os.path.join(dir_name, file))
df_tn.head(2)

,Rv_ID,Name,Insertions,# TA Sites,Length of Maximum Run,Span of Nucleotides,Sassetti-03,Post. Prob. Of Ess.,Call
0,Rv0001,dnaA,1,32,31,1365,1,1.0,E
1,Rv0002,dnaN,0,31,31,1167,-1,1.0,E


### DeJesus, Iorger (2017A)

#####  (you need to talk about this with Michael!!!!)

In [138]:
file_list = ['2017A_DeJesus_Iorger/table_1.xlsx']


____________
____________
____________
# OTHER stuff 

### Kieser Rubin (2015 data) - tables A, B, C

pre-formatting: 

In [109]:
# file_list = ['2015_Kieser_Rubin/table_1A.xlsx', 
#              '2015_Kieser_Rubin/table_1B.xlsx',
#              '2015_Kieser_Rubin/table_1C.xlsx']

# for file in file_list:
#     # read the data. 
#     df_tn = pd.read_excel(os.path.join(dir_name, file))

#     # this fixes the Rv_IDs for this particular dataset. 
#     df_tn = df_tn[~df_tn.Rv_ID.str.contains('IG')]
#     rv_id_list = [rv_id.split('_')[-1].strip('\'') for rv_id in df_tn['Rv_ID'].values]
#     df_tn['Rv_ID'] = rv_id_list

#     # write edited file:
#     outfile = file.split('.xlsx')[0]+'_edited.xlsx'
#     df_tn.to_excel(os.path.join(dir_name, outfile), index=False)

### FLUTE datasets pre-processing:

In [149]:
flute_path = 'Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/original'
file_list_flute = [os.path.join(flute_path, file) for file in os.listdir(flute_path)]

In [150]:
for file in file_list_flute:
    file_out = file.split('/')[-1].split('.dat')[0]+'.tsv'
    file_out = os.path.join('Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/', file_out)

    with open(file, 'r') as fin:
        lines_temp = fin.read()
        lines_new = lines_temp.split('\n')[6:]
        with open(file_out, 'w') as fout:
            fout.write('\n'.join(lines_new))

In [151]:
flute_path = 'Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/'
tsv_list_flute = [os.path.join(flute_path, file) for file in os.listdir(flute_path) if 'resampling' in file]

In [152]:
for file in tsv_list_flute:
    print(file)
    file_out = file.split('/')[-1].split('.tsv')[0]+'.xlsx'
    file_out = os.path.join('Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/', file_out)
    df_flute = pd.read_csv(file, sep='\t')
    df_flute.rename(columns={'#Orf':'Rv_ID','Name':'gene_name', 'p-adj':'Adj. p-value', 'log2 FC':'log2FC'}, inplace=True)
    df_flute.to_excel(file_out, index=False)

Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_WT_marP_s10000_pc0.00_1.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv0950_day0_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv3916c_day0_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv0307c_day0_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_vs_delta_Rv3717_TTR.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_glycerol_ponA1KO_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv1565c_day32_TTR.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv3684_day0_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_WT_Rv0954_s10000_pc0.00.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_vs_delta_Rv3811_TTR.tsv
Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv1096_day0_s10000_pc0.00

In [153]:
f_temp = 'Tn_library_studies/FLUTE_KO_TnSeq/FLUTE_log2fc/resampling_H37Rv_Rv1432_day32_TTR.xlsx'

In [154]:
df = pd.read_excel(f_temp)

In [155]:
df.head()

,Rv_ID,gene_name,Description,N,TAs Hit,Sum Rd 1,Sum Rd 2,Delta Rd,log2FC,p-value,Adj. p-value
0,Rv0001,dnaA,chromosomal replication initiation protein,31,1,0.2,0.0,-0.2,3.83,1.0000,1.00000
1,Rv0002,dnaN,DNA polymerase III subunit beta,31,0,0.0,0.0,0.0,0.00,1.0000,1.00000
2,Rv0003,recF,recombination protein F,35,9,19.1,54.0,34.9,1.50,0.3056,0.69974
3,Rv0004,-,hypothetical protein Rv0004,7,0,0.0,0.0,0.0,0.00,1.0000,1.00000
4,Rv0005,gyrB,DNA gyrase subunit B,42,3,35.2,1.0,-34.2,-5.08,0.1396,0.46795
